# 中间件：人在环路
<img src="./assets/LC_HITL.png" width="300">



## 设置

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 文件加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env("example.env")

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [ ]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [ ]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime


@tool
def execute_sql(query: str) -> str:
    """执行 SQLite 命令并返回结果。"""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db
    
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [ ]:
SYSTEM_PROMPT = """你是一个谨慎的 SQLite 分析师。

规则：
- 逐步思考。
- 当你需要数据时，使用一个 SELECT 查询调用工具 `execute_sql`。
- 只读操作；不要执行 INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE。
- 除非用户明确要求，否则限制为 5 行。
- 如果工具返回 'Error:'，修改 SQL 并重试。
- 优先使用明确的列列表；避免使用 SELECT *。
- 如果数据库离线，请要求用户稍后重试，不要进一步评论。
"""

## 中间件

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver(),
    context_schema=RuntimeContext,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"execute_sql": {"allowed_decisions": ["approve", "reject"]}},
        ),
    ],
)

In [ ]:
from langgraph.types import Command

question = "What are the names of all the employees?"

config = {"configurable": {"thread_id": "1"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
    context=RuntimeContext(db=db)
)

if "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m 中断：{description}\033[0m"
    )

    result = agent.invoke(
        Command(
            resume={
                "decisions": [{"type": "reject", "message": "数据库离线。"}]
            }
        ),
        config=config,  # 使用相同的线程 ID 恢复暂停的对话
        context=RuntimeContext(db=db),
    )
    print(f"\033[1;3;31m{80 * '-'}\033[0m")

print(result["messages"][-1].content)

In [ ]:
config = {"configurable": {"thread_id": "2"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
    context=RuntimeContext(db=db)
)

while "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m 中断：{description}\033[0m"
    )
    
    result = agent.invoke(
        Command(
            resume={"decisions": [{"type": "approve"}]}
        ),
        config=config,  # 使用相同的线程 ID 恢复暂停的对话
        context=RuntimeContext(db=db),
    )

for msg in result["messages"]:
    msg.pretty_print()